# Importação dos dados das proposições legislativas

Há duas formas principais para acessar os dados da ALMG. A primeira, mais simples, é o dowload direto dos arquivos csv estando atualmente disponíveis sete conjuntos: [**Legislação mineira**](https://dadosabertos.almg.gov.br/documentacao/arquivos/legislacao-mineira), [**Proposições**](https://dadosabertos.almg.gov.br/documentacao/arquivos/proposicoes), [**Tramitação de proposições**](https://dadosabertos.almg.gov.br/documentacao/arquivos/tramitacao-de-proposicoes), [**Deputados da atual legislatura**](https://dadosabertos.almg.gov.br/documentacao/arquivos/deputados-da-atual-legislatura), [**Deputados**](https://dadosabertos.almg.gov.br/documentacao/arquivos/deputados) desde a 8ª Legislatura e [**Verbas indenizatórias**](https://dadosabertos.almg.gov.br/documentacao/arquivos/verbas-indenizatorias).

A segunda forma é por meio da *API*, que permite consultas mais personalizadas, com resultados no formato XML ou JSON. Para este projeto de análse exploratória, utilizei arquivos csv e dados obtidos via API em formato JSON. Por exemplo, os dados de proposições foram baixados como CSV, enquanto, enquanto dados sobre legislaturas[^1] foram extraídos por meio da API. Todos os arquivos foram armazenados em um banco de dados *SQLite* para posterior análise.

[^1]: Incluem nesse conjunto a data de início, data de fim e data da eleição referente à cada legislatura.

In [1]:
#| code-fold: true
#| code-summary: "Código para importar dados de proposições"
#| eval: false

import requests
import sqlite3
import csv
from io import StringIO

def baixar_arquivo_em_memoria(ano):
    print(f"Baixando arquivos de proposições para o ano de {ano}")
    url = f"https://dadosabertos.almg.gov.br/arquivo/proposicoes/download?ano={ano}&tipo=CSV"
    response = requests.get(url)
    response.raise_for_status()
    print(f"Arquivo de {ano} baixado com sucesso na memória")
    return response.text

def criar_tabela_proposicoes(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS proposicoes (
            Codigo TEXT PRIMARY KEY,
            TipoProposicao TEXT,
            SiglaTipoProposicao TEXT,
            Numero TEXT,
            Ano TEXT,
            Ementa TEXT,
            Indexacao TEXT,
            Situacao TEXT,
            DataPublicacao TEXT,
            DataAtualizacao TEXT,
            DataUltimaAcao TEXT,
            Regime TEXT,
            Resumo TEXT,
            Origem TEXT,
            Local TEXT,
            NomeFaseAtual TEXT,
            Legislatura TEXT,
            Autores TEXT,
            LinkTextos TEXT,
            ano_arquivo INTEGER
        )
    ''')
    conn.commit()

def carregar_csv_no_sqlite(ano, conn):
    csv_text = baixar_arquivo_em_memoria(ano)
    csv_file = StringIO(csv_text)
    reader = csv.DictReader(csv_file, delimiter=',')

    cursor = conn.cursor()

    for row in reader:
        # Adiciona o ano na linha
        row['ano_arquivo'] = ano

        # Cria uma tupla com os valores na ordem correta
        values = (
            row.get('Codigo'),
            row.get('TipoProposicao'),
            row.get('SiglaTipoProposicao'),
            row.get('Numero'),
            row.get('Ano'),
            row.get('Ementa'),
            row.get('Indexacao'),
            row.get('Situacao'),
            row.get('DataPublicacao'),
            row.get('DataAtualizacao'),
            row.get('DataUltimaAcao'),
            row.get('Regime'),
            row.get('Resumo'),
            row.get('Origem'),
            row.get('Local'),
            row.get('NomeFaseAtual'),
            row.get('Legislatura'),
            row.get('Autores'),
            row.get('LinkTextos'),
            row.get('ano_arquivo')
        )

        # Executa o insert ou replace
        cursor.execute('''
            INSERT OR REPLACE INTO proposicoes (
                Codigo, TipoProposicao, SiglaTipoProposicao, Numero, Ano,
                Ementa, Indexacao, Situacao, DataPublicacao, DataAtualizacao,
                DataUltimaAcao, Regime, Resumo, Origem, Local, NomeFaseAtual,
                Legislatura, Autores, LinkTextos, ano_arquivo
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', values)

    conn.commit()
    print(f"Dados do ano {ano} adicionados/atualizados na tabela 'proposicoes'")

# Conectar ao banco SQLite
conn = sqlite3.connect('database.db')

# Criar tabela (se não existir)
criar_tabela_proposicoes(conn)

# Carregar dados de anos específicos
anos = range(1980, 2026)
for ano in anos:
    carregar_csv_no_sqlite(ano, conn)

# Fechar conexão
conn.close()

In [2]:
#| code-fold: true
#| code-summary: "Código para importar dados de legislaturas"
#| eval: false

import requests
import sqlite3

# URL do JSON
url = "https://dadosabertos.almg.gov.br/api/v2/legislaturas/lista?formato=json"

# Baixar o JSON
response = requests.get(url)
data = response.json()

# Conectar ao banco SQLite (cria se não existir)
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Criar tabela legislaturas
cursor.execute('''
CREATE TABLE IF NOT EXISTS legislaturas (
    id INTEGER PRIMARY KEY,
    dataInicio TEXT,
    dataTermino TEXT,
    dataEleicao TEXT
)
''')

# Inserir dados na tabela
for item in data.get('listaLegislatura', []):
    cursor.execute('''
    INSERT OR REPLACE INTO legislaturas (id, dataInicio, dataTermino, dataEleicao)
    VALUES (?, ?, ?, ?)
    ''', (
        item.get('id'),
        item.get('dataInicio'),
        item.get('dataTermino'),
        item.get('dataEleicao')  # Pode ser None se não existir
    ))

# Salvar (commit) e fechar conexão
conn.commit()
conn.close()

print("Dados inseridos com sucesso na tabela legislaturas.")

# Exploração dos dados

In [3]:
#| code-fold: true

# Importações
import pandas as pd
import sys
import os
import sqlite3
import locale

# Configurações
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

# Adiciona a raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Importações locais
from funcoes.utils import formatar_numero_brl
from funcoes.Figuras import create_horizontal_bar_chart_go

# Configuração do banco de dados
db_path = os.path.abspath(os.path.join(os.getcwd(), '../database.db'))

# Conexão e consultas
conn = sqlite3.connect(db_path)

# Carrega dados das proposições
df = pd.read_sql_query(
    "SELECT * FROM proposicoes", 
    conn
).sort_values('Ano', ascending=False, ignore_index=True)

# Carrega dados das legislaturas
legislaturas = pd.read_sql_query("SELECT * FROM legislaturas", conn)

# Fecha conexão
conn.close()

# Processamento dos dados
nr_proposicoes = formatar_numero_brl(len(df), casas_decimais=0)

legislaturas['dataInicio'] = pd.to_datetime(
    legislaturas['dataInicio'], 
    format='%Y-%m-%d'
)

inicio_legislaturas_ano = legislaturas['dataInicio'].dt.year

A análise exploratória começa pelas proposições[^2]. Os dados abrangem o período de 1959-2025, totalizando `{python} nr_proposicoes` registros. A @tbl-proposicao apresenta uma amostra destes dados. Entre as informações básicas estão o tipo, número e ano da proposição, o resumo (Ementa), temas (Indexacao) e a fase atual do processo legislativo.

[^2]: Uma "proposição" refere-se a qualquer matéria que está em tramitação na ALMG. Inclui diversos tipos de documentos como projetos de lei, projetos de lei complementar, propostas de emenda à constituição, projetos de resolução, vetos, indicações e requerimentos.

Na tabela de proposições destaco ainda a coluna "Autores", que contém um dicionário com a identificação dos parlamentares ou entidades participaram da autoria da proposição. Esta informação é útil para estudos de redes de co-autoria, permitindo identificar conexões políticas, sociais e possíveis hierarquias entre os parlamentares. 

Alémd disso, a tabela oferece ainda link para download dos textos completos das proposição. Esses arquivos possibilitam análises qualitativas, como a análise do discurso, e quantitativas por meio da mineração de texto. Isso permite identificar o perfil ideológico dos parlamentares, individualmente ou em grupos, e revelar os principais temas debatidos na Casa Legislativa.

## Contagem de proposições por tipo

In [4]:
#| label: tbl-proposicao
#| tbl-cap: Amostra dos dados sobre proposições legislativas
#| echo: false

import numpy as np

proposicao_tipo = pd.DataFrame({
    'count': df.TipoProposicao.value_counts(),
    'prop': df.TipoProposicao.value_counts(normalize=True)
})

# Substituir categorias com proporção < 1% por 'Outras'
new_index = np.where(proposicao_tipo['prop'] < 0.01, 'Outras', proposicao_tipo.index)

proposicao_tipo.index = new_index

# Agrupar somando os counts e props das categorias repetidas
proposicao_tipo = proposicao_tipo.groupby(proposicao_tipo.index).sum()

# Limpar e formatar categorias: strip e title case
proposicao_tipo.index = proposicao_tipo.index.str.strip().str.title()

# Ordenar
proposicao_tipo = proposicao_tipo.sort_values('count', ascending=True)

# Transformar index em coluna
proposicao_tipo = proposicao_tipo.reset_index().rename(columns={'index': 'TipoProposicao'})

nr_pl = proposicao_tipo.loc[proposicao_tipo['TipoProposicao'] == 'Projeto De Lei', 'count'].values[0]

prop_pl = proposicao_tipo.loc[proposicao_tipo['TipoProposicao'] == 'Projeto De Lei', 'prop'].values[0] * 100


nr_rn = proposicao_tipo.loc[proposicao_tipo['TipoProposicao'] == 'Requerimento Numerado', 'count'].values[0]

prop_rn = proposicao_tipo.loc[proposicao_tipo['TipoProposicao'] == 'Requerimento Numerado', 'prop'].values[0] * 100


nr_pl = formatar_numero_brl(nr_pl, casas_decimais=0)

prop_pl = formatar_numero_brl(prop_pl, casas_decimais=2)

nr_rn = formatar_numero_brl(nr_rn, casas_decimais=0)

prop_rn = formatar_numero_brl(prop_rn, casas_decimais=2)

df.head()

,Codigo,TipoProposicao,SiglaTipoProposicao,Numero,Ano,Ementa,Indexacao,Situacao,DataPublicacao,DataAtualizacao,DataUltimaAcao,Regime,Resumo,Origem,Local,NomeFaseAtual,Legislatura,Autores,LinkTextos,ano_arquivo
0,RQC 15024/2025,REQUERIMENTO DE COMISSÃO,RQC,15024,2025,None,/Tema/Administração Pública/Administração Esta...,Aprovado,28/06/2025,30/06/2025,25/06/2025,Votado nas comissões,None,None,Comissão de Educação Ciência e Tecnologia,None,20,"[{""id"":28869,""nome"":""Deputada Lohanna"",""partid...",https://dadosabertos.almg.gov.br/api/proposico...,2025
1,RQC 14967/2025,REQUERIMENTO DE COMISSÃO,RQC,14967,2025,None,None,Aprovado,28/06/2025,30/06/2025,25/06/2025,Votado nas comissões,None,None,Comissão de Educação Ciência e Tecnologia,None,20,"[{""id"":26098,""nome"":""Deputada Beatriz Cerqueir...",https://dadosabertos.almg.gov.br/api/proposico...,2025
2,RQC 15031/2025,REQUERIMENTO DE COMISSÃO,RQC,15031,2025,None,/Tema/Administração Pública/Patrimônio Público...,Aprovado,28/06/2025,30/06/2025,25/06/2025,Votado nas comissões,None,None,Comissão de Educação Ciência e Tecnologia,None,20,"[{""id"":26098,""nome"":""Deputada Beatriz Cerqueir...",https://dadosabertos.almg.gov.br/api/proposico...,2025
3,RQC 15032/2025,REQUERIMENTO DE COMISSÃO,RQC,15032,2025,None,/Tema/Educação/Estabelecimento de Ensino/Educa...,Aprovado,28/06/2025,30/06/2025,25/06/2025,Votado nas comissões,None,None,Comissão de Educação Ciência e Tecnologia,None,20,"[{""id"":26098,""nome"":""Deputada Beatriz Cerqueir...",https://dadosabertos.almg.gov.br/api/proposico...,2025
4,RQC 15033/2025,REQUERIMENTO DE COMISSÃO,RQC,15033,2025,None,/Tema/Educação/Estabelecimento de Ensino/Educa...,Aprovado,28/06/2025,30/06/2025,25/06/2025,Votado nas comissões,None,None,Comissão de Educação Ciência e Tecnologia,None,20,"[{""id"":26098,""nome"":""Deputada Beatriz Cerqueir...",https://dadosabertos.almg.gov.br/api/proposico...,2025


Para finalizar essa parte introdutória na @fig-tipo-proposicao uma contagem dos tipos de proposição. Importante destacar que, embora os projetos de lei tenham grande representatividade, respondendo com `{python} nr_pl` (cerca de `{python} prop_pl`), eles são superados pelos Requerimentos Numerados, que representam o tipo de proposição mais recorrente, com `{python} nr_rn` (cerca de `{python} prop_pl`% do total).

Essa informação é relevante, considerando que a literatura frequentemente foca os projetos de lei como principal objeto de análise. É o caso por exemplo, de Fowler [-@fowler2006] em que a rede de coautoria é construída com base nos projetos de lei, a fim de gerar indicadores de influência legislativa. Considero que, embora a proposição de projetos de lei seja um importante indicativo da atuação
parlamentar e de seu perfil ideológico, não se deve negligenciar os demais tipos de proposição pois estes podem apresentar outras perspectivas da atividade legislativa.

In [5]:
#| code-fold: true
#| label: fig-tipo-proposicao
#| fig-cap: Contagem de proposições por tipo
#| echo: false

# Plotar gráfico horizontal
fig = create_horizontal_bar_chart_go(
    proposicao_tipo, 'count', 'TipoProposicao',
    title='', show_percentages=True, bar_color='blue'
)

fig.show()


## Evolução temporal do protocolo de proposições

Vamos observar agora a evolução das proposições legislativas ao longo do tempo. Haveria algum período específico em que os deputados e deputadas apresentam maior propensão a protocolar projetos de lei ou outros requerimentos? Para responder a essa pergunta, analisei os dados com base na data de protocolo das proposições legislativas. Não apliquei filtros quanto ao tipo de proposição nem quanto ao período, visando a uma
visualização ampla e geral.

A contagem foi agrupada mensalmente com a contagem bruta e média móvel de 03 meses estão na @fig-evolucao-temporal. Também inseri linhas verticais pontilhadas indicando o início de cada legislatura.

Destaco como principal ponto o pico de proposições no primeiro ano de cada legislatura, um fenômeno que se torna visualmente mais evidente a partir de 1987. Isso revela uma interessante avenida de pesquisa: por meio de métodos quantitativos, seria possível realizar análises estatísticas de séries temporais para identificar picos e sazonalidades de forma mais robusta, além de investigar possíveis causas para essas variações. Alternativamente, abordagens qualitativas poderiam explorar através da análise as razões para o aumento de proposições no primeiro ano de mandato.

In [6]:
#| code-fold: true
#| label: fig-evolucao-temporal
#| fig-cap: Número de proposições mensais
#| echo: false

import numpy as np
import plotly.graph_objects as go
from funcoes.Figuras import create_line_chart_go, create_time_series_chart_go
from funcoes.utils import preprocessar_proposicao_media_movel, preprocessar_proposicao_media_movel_mensal

# Checar valores nulos

valores_nulos_data = df[df['DataPublicacao'].isnull()]

nr_valores_nulos_data = formatar_numero_brl(len(valores_nulos_data),casas_decimais=0)

df['data'] = np.where(df['DataPublicacao'].notnull(), df['DataPublicacao'], df['DataAtualizacao'])

df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

proprosicao_media_movel = preprocessar_proposicao_media_movel_mensal(df, window=3)

# Step 2: Create chart using the new function
fig_media_movel = create_time_series_chart_go(
    data=proprosicao_media_movel,
    show_slider=True,
    date_col='month',
    value_col='total_propositions',
    ma_col='moving_avg',
    title='',
    extra_months=1,
    height=500
)

# Adiciona linhas tracejadas verticais para as legislaturas
for i, row in legislaturas.iterrows():
    data_inicio = row['dataInicio']
    
    if data_inicio >= pd.Timestamp('1959-01-01'):
        fig_media_movel.add_vline(
            x=data_inicio,
            line=dict(color="red", width=1.5, dash="dash"),
            layer="below"
        )

# Adiciona item na legenda para "Início de Legislatura"
fig_media_movel.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color="red", width=1.5, dash="dash"),
        name="Início de Legislatura"
    )
)


fig_media_movel.show()